# Data Cleaning and Validation

In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('WELLCOME/WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")

In [37]:
df.columns = ['PMID', 'Publisher', 'Journal', 'Article', 'Cost']

## five most common journals and the total articles for each

In [79]:
df['Journal_clean'] = df.Journal.apply(lambda x: str(x).lower().strip('\n').rstrip().lstrip().replace('&', 'and'))

The following code was done both series-wise, and entry-wise.  The entry-wise version returns better results.  It's a mystery.

In [80]:
df['Journal_clean'].str.replace('the ', '')
df['Journal_clean'].str.replace('bmj', 'biomedical journal')
df['Journal_clean'].str.replace('  ', ' ')
df['Journal_clean'].str.replace('.', '')
df['Journal_clean'].str.replace(' 1', ' one')
df['Journal_clean'].str.replace('j ', 'journal ')
df['Journal_clean'].str.replace('jnl ', 'journal ')
df['Journal_clean'].str.replace('am ', 'american')
df['Journal_clean'].str.replace('eur ', 'european')
df['Journal_clean'].str.replace('int ', 'international ')
df['Journal_clean'].str.replace('asn ', 'asian ')
df['Journal_clean'].str.replace('brt ', 'british ')
df['Journal_clean'].str.replace('trop ', 'tropical ') 
df['Journal_clean'].str.replace('bba ', 'biochimica et bioohysica acta')
df['Journal_clean'].str.replace('biochem ', 'biochemical ')
df['Journal_clean'].str.replace('mol ', 'molecular ')
df['Journal_clean'].str.replace('soc ', 'society ')
df['Journal_clean'].str.replace('trans ', 'transactions ')
# df['Journal_clean'].apply(lambda x: x.replace(' j', ' journal') if x[-2:] ==' j')
# df['Journal_clean'].apply(lambda x: 'proceedings of national academy of sciences' if all(word in x for word in ['proc', 'nat', 'acad', 'sci']) or ('pnas' in x))
# df['Journal_clean'].apply(lambda x: 'journal of biological chemistry' if all(word in journal for word in ['journal', 'biol',  'chem']) and (len(journal_words)<5))
# df['Journal_clean'].apply(lambda x: 'plos one' if all(word in journal for word in ['plos', 'one']))
# df['Journal_clean'].apply(lambda x: 'nucleic acids research' if all(word in journal for word in ['nucl', 'acid', 'res']))

for ik in range(len(df['Journal_clean'])):
    journal = df['Journal_clean'][ik]
    journal_words = journal.split()
    if df['Journal_clean'][ik][-2:] == ' j':
        df['Journal_clean'][ik] = df['Journal_clean'][ik].replace(' j', ' journal')
    if all(word in journal for word in ['proc', 'nat', 'acad', 'sci']) or ('pnas' in journal):
        df['Journal_clean'][ik] = 'proceedings of national academy of sciences'
    if all(word in journal for word in ['journal', 'biol',  'chem']) and (len(journal_words)<5) :
        df['Journal_clean'][ik] = 'journal of biological chemistry'
    if all(word in journal for word in ['plos', 'one']):
        df['Journal_clean'][ik] = 'plos one'
    if all(word in journal for word in ['nucl', 'acid', 'res']):
        df['Journal_clean'][ik] = 'nucleic acids research'
    df['Journal_clean'][ik] = df['Journal_clean'][ik].rstrip()


df['Journal_clean'].str.strip()



0                                  psychological medicine
1                                       biomacromolecules
2                                              j med chem
3                                              j med chem
4                                              j org chem
5                          journal of medicinal chemistry
6                            journal of proteome research
7                                               mol pharm
8                                    acs chemical biology
9                                    acs chemical biology
10           journal of chemical information and modeling
11                         journal of medicinal chemistry
12                         journal of medicinal chemistry
13                                           biochemistry
14                         journal of medicinal chemistry
15                                       gastroenterology
16                        journal of biological chemistry
17            

In [81]:
top_five = df['Journal_clean'].value_counts()[0:5]
print(top_five)

plos one                                       200
journal of biological chemistry                 55
proceedings of national academy of sciences     37
nucleic acids research                          29
neuroimage                                      29
Name: Journal_clean, dtype: int64


In [82]:
# for reasons I can't tease out, the entry-wise version resolves more errors

for ik in range(len(df['Journal_clean'])):
    journal = df['Journal_clean'][ik]
    journal_words = journal.split()
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('the ', '')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('bmj', 'biomedical journal')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('  ', ' ').replace('.', '')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace(' 1', ' one')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('j ', 'journal ').replace('jnl ', 'journal ')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('am ', 'american').replace('eur ', 'european').replace('int ', 'international ').replace('asn ', 'asian ').replace('brt ', 'british ').replace('trop ', 'tropical ') 
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('bba ', 'biochimica et bioohysica acta')
    df['Journal_clean'][ik] = df['Journal_clean'][ik].replace('biochem ', 'biochemical ').replace('mol ', 'molecular ').replace('soc ', 'society ').replace('trans ', 'transactions ')
    
    if df['Journal_clean'][ik][-2:] == ' j':
        df['Journal_clean'][ik] = df['Journal_clean'][ik].replace(' j', ' journal')
    if all(word in journal for word in ['proc', 'nat', 'acad', 'sci']) or ('pnas' in journal):
        df['Journal_clean'][ik] = 'proceedings of national academy of sciences'
    if all(word in journal for word in ['journal', 'biol',  'chem']) and (len(journal_words)<5) :
        df['Journal_clean'][ik] = 'journal of biological chemistry'
    if all(word in journal for word in ['plos', 'one']):
        df['Journal_clean'][ik] = 'plos one'
    if all(word in journal for word in ['nucl', 'acid', 'res']):
        df['Journal_clean'][ik] = 'nucleic acids research'
    df['Journal_clean'][ik] = df['Journal_clean'][ik].rstrip()



In [83]:
top_five = df['Journal_clean'].value_counts()[0:5]
print(top_five)

plos one                                       207
journal of biological chemistry                 62
proceedings of national academy of sciences     37
nucleic acids research                          29
neuroimage                                      29
Name: Journal_clean, dtype: int64


## calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [10]:
def parse_cost(x):
    try:
        return x.split('£')[1]
    except:
        try: 
            return x.strip('$')
        except:
            print(x)

In [32]:
def parse_cost(x):
    if '£' in x:
        cost = float(x.split('£')[1])
    elif '$' in x:
        cost = float(x.strip('$'))
    else:
        try:
            cost = float(x)
        except:
            print(x)
    if cost< 100000:
        return cost
    else:
        return np.nan

In [33]:
import numpy as np
df['Cost_clean'] = df.Cost.apply(parse_cost)#.apply(lambda x: float(x) if x != '999999.00' else np.nan ).apply(replace_extrema) 

In [20]:
def replace_extrema(x):
    if x > 100000:
        return np.nan
    else:
        return x

In [34]:
for ik in range(len(top_five)):
    sub_df = df[(df.Journal_clean == top_five.index[ik])]
    print(top_five.index[ik],'\n', sub_df.describe(), '\n')

plos one 
         Cost_clean
count   198.000000
mean    934.982121
std     194.966229
min     122.310000
25%     850.595000
50%     896.975000
75%    1040.125000
max    1785.360000 

journal of biological chemistry 
         Cost_clean
count    61.000000
mean   1402.140492
std     390.360651
min     265.670000
25%    1152.720000
50%    1311.730000
75%    1586.360000
max    2501.070000 

proceedings of national academy of sciences 
         Cost_clean
count    36.000000
mean    772.919167
std     370.191932
min     206.320000
25%     623.707500
50%     732.010000
75%     792.255000
max    2691.680000 

nucleic acids research 
         Cost_clean
count    29.000000
mean   1162.344828
std     442.150934
min     710.000000
25%     852.000000
50%     852.000000
75%    1704.000000
max    2184.000000 

neuroimage 
         Cost_clean
count    29.000000
mean   2215.168276
std     266.653947
min    1747.160000
25%    2100.540000
50%    2326.430000
75%    2396.570000
max    2503.340000 

